In [30]:
import json
from pathlib import Path
from pprint import pprint

from config import EMBEDDING_MODEL
from dotenv import find_dotenv, load_dotenv
from qdrant_client.http.models import PointStruct
from utils import (
    create_collection,
    embed_text,
    get_collection_info,
    get_count,
    search,
    upsert,
)

load_dotenv(find_dotenv())

True

In [14]:
raw_data_path = Path("../scraper/srb_labor_law_data.json")

In [15]:
with open(raw_data_path, "r", encoding="utf-8") as file:
    raw_data = json.loads(file.read())

## Embedd data

Create JSONL for parallel embedding

In [42]:
filename = Path("./requests_to_parallel_process.jsonl")
jobs = [
    {
        "model": EMBEDDING_MODEL,
        "input": ". ".join([sample["title"], " ".join(sample["texts"])]),
    }
    for sample in raw_data
]
with open(filename, "w") as f:
    for job in jobs:
        json_string = json.dumps(job)
        f.write(json_string + "\n")

In [ ]:
! python api_request_parallel_processor.py \
  --requests_filepath requests_to_parallel_process.jsonl \
  --save_filepath requests_to_parallel_process_results.jsonl \
  --request_url https://api.openai.com/v1/embeddings \
  --max_requests_per_minute 2500 \
  --max_tokens_per_minute 900000 \
  --token_encoding_name cl100k_base \
  --max_attempts 5 \
  --logging_level 20

Create PointStructures for Qdrant database

In [11]:
embeddings_path = Path("./requests_to_parallel_process_results.jsonl")
with open(embeddings_path, "r", encoding="utf-8") as file:
    embeddings = []
    for line in file:
        embeddings.append(json.loads(line))

In [12]:
embeddings_lookup = {}
for item in embeddings:
    text = item[0]["input"]
    article_name = text.split(". ")[0]
    embedding = item[1]["data"][0]["embedding"]
    embeddings_lookup[article_name] = {"embedding": embedding, "text": text}

In [16]:
points = []

for id, dictionary in enumerate(raw_data):
    title = dictionary["title"]
    link = dictionary["link"]
    if title in embeddings_lookup:
        embedding, text = (
            embeddings_lookup[title]["embedding"],
            embeddings_lookup[title]["text"],
        )
        points.append(
            PointStruct(
                id=id,
                vector=embedding,
                payload={"title": title, "text": text, "link": link},
            )
        )
    else:
        print(
            f"Warning: No embedding found for title '{title}'. This item will be skipped."
        )

# Create Vector database

In [17]:
collection_name = "labor_law"
create_collection(name=collection_name)

2024-03-23 22:15:21.809 | INFO     | utils:create_collection:30 - Creating collection: labor_law with vector size: 1536.


True

In [18]:
upsert(collection=collection_name, points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [19]:
get_collection_info(collection=collection_name)

{'status': <CollectionStatus.GREEN: 'green'>,
 'optimizer_status': <OptimizersStatusOneOf.OK: 'ok'>,
 'vectors_count': 313,
 'indexed_vectors_count': 0,
 'points_count': 313,
 'segments_count': 2,
 'config': {'params': {'vectors': {'size': 1536,
    'distance': <Distance.COSINE: 'Cosine'>,
    'hnsw_config': None,
    'quantization_config': None,
    'on_disk': None},
   'shard_number': 1,
   'sharding_method': None,
   'replication_factor': 1,
   'write_consistency_factor': 1,
   'read_fan_out_factor': None,
   'on_disk_payload': True,
   'sparse_vectors': None},
  'hnsw_config': {'m': 16,
   'ef_construct': 100,
   'full_scan_threshold': 10000,
   'max_indexing_threads': 0,
   'on_disk': False,
   'payload_m': None},
  'optimizer_config': {'deleted_threshold': 0.2,
   'vacuum_min_vector_number': 1000,
   'default_segment_number': 0,
   'max_segment_size': None,
   'memmap_threshold': None,
   'indexing_threshold': 20000,
   'flush_interval_sec': 5,
   'max_optimization_threads': None

In [20]:
get_count(collection=collection_name)

313

# Search the Vector database 

In [32]:
path_to_tests = Path("./test_queries.json")
with open(path_to_tests, "r", encoding="utf-8") as file:
    test_samples = json.loads(file.read())

In [34]:
test_samples["hard"]

[{'query': 'Koliko dana godišnjeg odmora zaposleni u Srbiji ima po zakonu?',
  'answer': 'U kalendarskoj godini zaposleni ima pravo na godišnji odmor u trajanju utvrđenom opštim aktom i ugovorom o radu, a najmanje 20 radnih dana.'},
 {'query': 'Da li maloletno lice može zasnovati radni odnos?',
  'answer': 'Može, ako ima preko 15 godina, ali samo na poslovima koji ne ugrožavaju zdravlje, moral i obrazovanje maloletnika, na osnovu nalaza zdravstvenog organa kojim se utvrđuje da je sposobno za obavljanje poslova i da takvi poslovi nisu štetni za zdravlje maloletnika, uz pisanu saglasnost roditelja, usvojioca ili staraoca maloletnika.'},
 {'query': 'Da li zaposleni ima pravo na veću zaradu ukoliko radi na neradni dan?',
  'answer': 'Zaposleni ima pravo na uvećanje zarade za rad tokom praznika koji se slavi neradno - u kom slučaju ostvaruje pravo na najmanje 110% ugovorene osnovne zarade, ako radi noću - kad ima pravo na najmanje 26% od ugovorene osnovne zarade, ili prekovremeno - kad tako

Get embeddings for tests

In [36]:
for level in test_samples.keys():
    for i, sample in enumerate(test_samples[level]):
        response = embed_text(text=sample["query"])
        embedding = response.data[0].embedding
        test_samples[level][i]["embedding"] = embedding

Save tests with embeddings

In [44]:
with open(path_to_tests, "w", encoding="utf-8") as file:
    file.write(json.dumps(test_samples, indent=4))

In [41]:
query = test_samples["hard"][3]["query"]
embedding = test_samples["hard"][3]["embedding"]
query

'Koliko traje porodiljsko odsustvo?'

In [42]:
response = search(collection=collection_name, query_vector=embedding, with_vectors=True)

In [43]:
pprint([point.payload for point in response])

[{'link': 'https://www.paragraf.rs/propisi/zakon_o_radu.html#clan_94',
  'text': 'Član 94. Zaposlena žena ima pravo na odsustvo sa rada zbog trudnoće '
          'i porođaja (u daljem tekstu: porodiljsko odsustvo), kao i odsustvo '
          'sa rada radi nege deteta, u ukupnom trajanju od 365 dana. Zaposlena '
          'žena ima pravo da otpočne porodiljsko odsustvo na osnovu nalaza '
          'nadležnog zdravstvenog organa najranije 45 dana, a obavezno 28 dana '
          'pre vremena određenog za porođaj. Porodiljsko odsustvo traje do '
          'navršena tri meseca od dana porođaja. Zaposlena žena, po isteku '
          'porodiljskog odsustva, ima pravo na odsustvo sa rada radi nege '
          'deteta do isteka 365 dana od dana otpočinjanja porodiljskog '
          'odsustva iz stava 2. ovog člana. Otac deteta može da koristi pravo '
          'iz stava 3. ovog člana u slučaju kad majka napusti dete, umre ili '
          'je iz drugih opravdanih razloga sprečena da koristi to p